<img style="float: left;;" src='Figures/iteso.jpg' width="50" height="100"/></a>

# <center> <font color= #000047> Discretización por Clustering Jerárquico



El clustering jerárquico puede utilizarse para **discretizar** variables numéricas, es decir, agrupar valores continuos en categorías o intervalos. Esto es útil para:
- Transformar variables continuas en variables categóricas para modelos que requieren categorías.
- Descubrir agrupamientos naturales en los datos sin definir previamente los límites de los intervalos.

El clustering jerárquico es una técnica de agrupamiento no supervisado que construye una jerarquía de clusters. Existen dos enfoques principales:
- **Aglomerativo (bottom-up):** Cada punto inicia como un cluster y se van fusionando sucesivamente.
- **Divisivo (top-down):** Todos los puntos inician en un solo cluster y se van dividiendo.

El método más común es el aglomerativo.


1. **Matriz de distancias:** Se calcula la distancia entre todos los pares de puntos. Por ejemplo, la distancia euclidiana:
$$
d(x_i, x_j) = \sqrt{\sum_{k=1}^p (x_{ik} - x_{jk})^2}
$$

2. **Criterios de enlace (linkage):** Determinan cómo se calcula la distancia entre clusters:

|Método               |                         | 
|:-------------------:|:------------------------------------------|
|**Complete**             | $d(u,v) = \max(d(u[i],k[i]),d(v[i],k[i]))$ |
|**Single**               | $d(u,v) = \min(d(u[i],k[i]),d(v[i],k[i]))$ |
|**Average**              | $d(u,v) = \frac{n_u d(u[i],k[i]) + n_v d(v[i],k[i])}{n_u + n_v}$|
|**Centroid**             | $d(u,v) = ||c_u - c_v ||_2 = \sqrt(\frac{n_u d(u[i],k[i]) + n_v d(v[i],k[i])}{n_u + n_v} - \frac{n_u n_v d(u[i],v[i])}{(n_u + n_v)^2})$                      |
|**Ward**                 | $d(u,v) = \sqrt(\frac{n_u d(u[i],k[i]) + (n_v + n_k) d(v[i],k[i]) - n_k d(u[i],v[i])}{n_u + n_v + n_k})$                      |

3. **Algoritmo aglomerativo:**
   1. Cada punto es un cluster.
   2. Calcular la matriz de distancias entre clusters.
   3. Fusionar los dos clusters más cercanos.
   4. Repetir hasta que quede un solo cluster o se alcance el número deseado.


## Ejemplo de Clustering Jerárquico

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

#Método para calcular los clusters usando el algoritmo de Clustering Jerárquico
from scipy.cluster import hierarchy

In [ ]:
np.random.seed(100)
a = np.random.multivariate_normal([10,10], [[3,0],[0,3]], size=[100])
b = np.random.multivariate_normal([0,20], [[3,0],[0,3]], size=[100])
c = np.random.multivariate_normal([20,20], [[3,0],[0,3]], size=[100])

x = np.concatenate((a,b,c))

In [ ]:
len(x)

In [ ]:
x

In [ ]:
plt.figure(figsize=(6,4))
plt.scatter(x[:,0], x[:,1])
plt.xlabel('$x1$')
plt.ylabel('$x2$')
plt.grid()
plt.show()

In [ ]:
help(hierarchy)

In [ ]:
help(hierarchy.linkage)

In [ ]:
Z = hierarchy.linkage(x, metric='euclidean', method= 'ward')

In [ ]:
len(Z)

In [ ]:
pd.DataFrame(Z)

In [ ]:
help(hierarchy.dendrogram)

## Criterios de selección de grupos


In [ ]:
plt.figure(figsize=(10,8))
dn = hierarchy.dendrogram(Z)

plt.title('Dendograma Completo')
plt.xlabel('Indices de las muestras')
plt.ylabel('Distancias de similitud')
plt.show()

In [ ]:
plt.figure(figsize=(10,8))
dn = hierarchy.dendrogram(Z, truncate_mode ='level', p=3)

plt.title('Dendograma Completo')
plt.xlabel('Indices de las muestras')
plt.ylabel('Distancias de similitud')
plt.show()

In [ ]:
grupos_opt = 3
grupos_datos = hierarchy.fcluster(Z,grupos_opt, criterion='maxclust')
# Como tarea para investigar, qué hace el método fcluster ()
grupos_datos

In [ ]:
len(grupos_datos)

In [ ]:
X = pd.DataFrame(x, columns=['x1','x2'])
X['cluster'] = grupos_datos

In [ ]:
X

In [ ]:
plt.figure(figsize=(6,4))
plt.scatter(X['x1'].values, X['x2'].values, c=X['cluster'].values)
plt.xlabel('$x1$')
plt.ylabel('$x2$')
plt.grid()
plt.show()

### Criterio del Codo (1er criterio)


In [ ]:
pd.DataFrame(Z)

In [ ]:
last = Z[-15:,2]
last_rev = last[::-1]
indx_group = np.arange(1, len(last_rev)+1)
indx_group

In [ ]:
last_rev

In [ ]:
#Grafica del criterio del codo
plt.plot(indx_group, last_rev)
plt.xlabel('Número de grupos')
plt.ylabel('Inercia de las distancias entre grupos')
plt.grid()
plt.show()

In [ ]:
#El número de grupos opt mediante el criterio del codo es 3
grupos_opt = 3
grupos_datos = hierarchy.fcluster(Z,grupos_opt, criterion='maxclust')
# Como tarea para investigar, qué hace el método fcluster ()
grupos_datos

In [ ]:
plt.figure(figsize=(6,4))
plt.scatter(X['x1'].values, X['x2'].values, c=X['cluster'].values)
plt.xlabel('$x1$')
plt.ylabel('$x2$')
plt.grid()
plt.show()


In [ ]:
def criterio_codo(Z, n_grupos):
    last = Z[-n_grupos:,2]
    last_rev = last[::-1]
    indx_group = np.arange(1,len(last_rev)+1)
    
    #Gráfica del codo
    plt.plot(indx_group, last_rev)
    plt.xlabel('Número de grupos')
    plt.ylabel('Inercia de las distancias entre grupos')
    plt.grid()
    plt.show()

In [ ]:
criterio_codo(Z, 30)

### Criterio del gradiente


In [ ]:
last = Z[-6:,2]
gradiente = np.diff(last)
grad_rev = gradiente[::-1]

indx_group = np.arange(2,len(grad_rev)+2)

#Graficar el criterio del gradiente
plt.plot(indx_group, grad_rev)
plt.xlabel('Núemro de grupos')
plt.ylabel('Gradiente de la inercia en las distancias entre grupos')
plt.grid()
plt.show()

In [ ]:
def criterio_gradiente(Z, n_grupos):
    last = Z[-n_grupos:,2]
    gradiente = np.diff(last)
    grad_rev = gradiente[::-1]

    indx_group = np.arange(2,len(grad_rev)+2)

    #Graficar el criterio del gradiente
    plt.plot(indx_group, grad_rev)
    plt.xlabel('Número de grupos')
    plt.ylabel('Gradiente de la inercia en las distancias entre grupos')
    plt.grid()
    plt.show()

In [ ]:
criterio_gradiente(Z, 5)

## Aplicar número de grupos opt

In [ ]:
#El número de grupos opt mediante el criterio del codo es 3
grupos_opt = 3
grupos_datos = hierarchy.fcluster(Z,grupos_opt, criterion='maxclust')
# Como tarea para investigar, qué hace el método fcluster ()
grupos_datos

In [ ]:
plt.figure(figsize=(6,4))
plt.scatter(X['x1'].values, X['x2'].values, c=X['cluster'].values)
plt.xlabel('$x1$')
plt.ylabel('$x2$')
plt.grid()
plt.show()

## Ejemplo 2: 

Consideremos el datasety `shopping-data.csv`, este dataset contiene información sobre el ingreso anual y gastos de clientes de una empresa.

Conservaremos las columnas Ingresos anuales (en miles de dólares) y Puntuación de gastos (1-100). La columna Spending Score indica la frecuencia con la que una persona gasta dinero en un centro comercial en una escala del 1 al 100, siendo 100 el que más gasta.

In [ ]:
data = pd.read_csv('shopping-data.csv')

In [ ]:
data.head()

In [ ]:
data_split = data.iloc[:,-2:]

In [ ]:
data_split.head()

In [ ]:
data_split.values

In [ ]:
# Visualizar los datos Anual income vs Spending Score
plt.figure(figsize=(6,4))
plt.scatter(data_split.values[:,0],data_split.values[:,1])
plt.xlabel('Anual Income (k$)')
plt.ylabel('Spending Score (1-100)')
plt.grid()
plt.show()

In [ ]:
#Aplicar el clustering Jerárquico (Elegir el criterio de linkage, metrica de similitud)
#Decidir cuántos grupos seleccionar para la clusterización
# - Dendrogramna
# - C. codo
# - C. Gradiente
# Graficar los datos con la clusterización (con sus grupos correspondientes)


In [ ]:
Z = hierarchy.linkage(data_split.values, metric='euclidean', method='ward')

In [ ]:
pd.DataFrame(Z)

In [ ]:
# Dendrograma
plt.figure(figsize=(10,8))
dn = hierarchy.dendrogram(Z)
plt.title('Dendrograma completo')
plt.xlabel('Ind. de las muestras')
plt.ylabel('Distancia de similitud')
plt.show()

In [ ]:
#Con el dendrograma de arriba se puede conlcuir que podemos agrupar con 5 grupos

In [ ]:
#Criterio del codo
criterio_codo(Z, 15)

In [ ]:
# Con el criterio del codo se puede concluir que tenemos 5 o 7 grupos

In [ ]:
#Criterio del codo
criterio_gradiente(Z, 15)

In [ ]:
# Con el criterio del gradiente se puede observar que se puede clusterizar con 3, 5 o 7, 8

In [ ]:
grup_opt =3
grupos_datos = hierarchy.fcluster(Z, grup_opt, criterion='maxclust')


In [ ]:
# Visualizar los datos Anual income vs Spending Score
plt.figure(figsize=(6,4))
plt.scatter(data_split.values[:,0],data_split.values[:,1], c=grupos_datos)
plt.xlabel('Anual Income (k$)')
plt.ylabel('Spending Score (1-100)')
plt.grid()
plt.show()

## Discretización de datos usando clustering jerárquico

**Procedimiento:**
1. Aplicar clustering jerárquico sobre la variable o variables de interés (puede ser univariado o multivariado).
2. Cortar el dendrograma en el número deseado de clusters (bins).
3. Asignar a cada dato la etiqueta de su cluster, que actúa como la categoría discreta.

#### Ejemplo: Discretización de una variable continua
Supongamos que queremos discretizar la variable `petal length (cm)` del dataset Iris en 4 categorías usando clustering jerárquico.

In [ ]:
from sklearn.datasets import load_iris
# Cargar datos
iris = load_iris()
X = iris.data
labels = iris.target

In [ ]:
# Usar clustering jerárquico para discretizar petal length
petal_length = X[:, 2].reshape(-1, 1)
Z_petal = hierarchy.linkage(petal_length, method='ward')
petal_bins = hierarchy.fcluster(Z_petal, t=4, criterion='maxclust')

plt.figure(figsize=(8,4))
plt.scatter(petal_length, np.zeros_like(petal_length), c=petal_bins, cmap='tab10', s=50)
plt.xlabel('Petal length (cm)')
plt.yticks([])
plt.title('Discretización de petal length usando clustering jerárquico')
plt.show()



- La discretización jerárquica puede ser útil como preprocesamiento para modelos que funcionan bien con variables categóricas (árboles, Naive Bayes, reglas, etc.).
- Puede ayudar a capturar patrones no lineales o a reducir el sobreajuste en variables continuas con outliers.